In [ ]:
import json
import os
from pprint import pprint

import pyomo.environ as pe
import pyomo.opt as po
import yaml

In [ ]:
cfl_traditional_filename = os.path.join('data', 'cfl.yaml')
cfl_blk_demand_filename = os.path.join('data', 'cfl_blk_by_demand.json')
cfl_blk_supply_filename = os.path.join('data', 'cfl_blk_by_supply.json')

# Setup Solver

In [ ]:
solver = po.SolverFactory('glpk')

# Capacitated Facility Location
To demonstrate blocks, let's setup and solve an instance of the CFL model.

Sets:
- $I$: supply sites, indexed by $i$
- $J$: demand sites, indexed by $j$

Parameters:
- $s_i$: supply capacity of supply site $i$
- $d_j$: demand required by demand site $j$
- $f_i$: fixed cost to open supply site $i$
- $c_{ij}$: variable cost to transport from supply site $i$ to demand site $j$

Variables:
- $x_{ij}$ - quantity of product to ship from supply site $i$ to demand site $j$
- $y_i$ - 0/1 decision variable indicating that supply site $i$ is producing

Model:
$$
\begin{alignat*}{3}
\text{minimize  }  & \sum_{i \in I} f_i y_i + \sum_{i \in I} \sum_{j \in J} c_{ij} x_{ij} \\
\text{subject to  }
& \sum_{i \in I} x_{ij} \ge d_j && \forall j \in J \\
& \sum_{j \in J} x_{ij} \le s_i y_i && \forall i \in I \\
& x \in \mathbb{R}_+^{|I| \times |J|} && \\
& y \in \{0, 1\}^{|I|} && \\
\end{alignat*}
$$

# Implementation without Blocks

In [ ]:
with open(cfl_traditional_filename) as fh:
    data = yaml.load(fh, Loader=yaml.FullLoader)
pprint(data)

In [ ]:
model = pe.ConcreteModel("Traditional")

# sets
model.I = pe.Set(initialize=data['supply_sites'])
model.J = pe.Set(initialize=data['demand_sites'])

# parameters
model.s = pe.Param(model.I, initialize=data['supply'])
model.d = pe.Param(model.J, initialize=data['demand'])
model.f = pe.Param(model.I, initialize=data['fixed_cost'])
model.c = pe.Param(model.I, model.J, initialize=data['variable_cost'])

# variables
model.x = pe.Var(model.I, model.J, domain=pe.NonNegativeReals)
model.y = pe.Var(model.I, domain=pe.Binary)

# constraints
def con_satisfaction(model, j):
    return sum(model.x[i, j] for i in model.I) >= model.d[j]
model.con_satisfaction = pe.Constraint(model.J, rule=con_satisfaction)

def con_transportation(model, i):
    return sum(model.x[i, j] for j in model.J) <= model.s[i] * model.y[i]
model.con_transportation = pe.Constraint(model.I, rule=con_transportation)

# objective
def obj_min_cost(model):
    return sum(model.f[i] * model.y[i] for i in model.I)\
        + sum(model.c[i, j] * model.x[i, j] for i in model.I for j in model.J)
model.obj_min_cost = pe.Objective(sense=pe.minimize, rule=obj_min_cost)

In [ ]:
result = solver.solve(model)
model.display()

# Implementation with Blocks Indexed by Demand Sites

In [ ]:
with open(cfl_blk_demand_filename) as fh:
    data = json.load(fh)
pprint(data)

In [ ]:
model = pe.ConcreteModel("Blocks (by Demand Site)")

# sets
model.I = pe.Set(initialize=data['supply_sites'])
model.J = pe.Set(initialize=data['demand_sites'])

# parameters (not indexed in J)
model.s = pe.Param(model.I, initialize=data['supply'])
model.f = pe.Param(model.I, initialize=data['fixed_cost'])

# variables (not indexed in J)
model.y = pe.Var(model.I, domain=pe.Binary)

# blocks (indexed in J)
def blk_demand(block, j):
    blk_data = data['demand_data'][j]
    I = block.model().I # borrow the set of supply sites from the overarching model
    block.d = pe.Param(initialize=blk_data['demand'])
    block.c = pe.Param(I, initialize=blk_data['variable_cost'])
    block.x = pe.Var(I, domain=pe.NonNegativeReals)
    block.con_satisfaction = pe.Constraint(expr=(sum(block.x[i] for i in I) >= block.d))
    block.variable_cost = sum(block.c[i] * block.x[i] for i in I)
model.blk_demand = pe.Block(model.J, rule=blk_demand)

# constraints (not indexed in J)
def con_transportation(model, i):
    return sum(model.blk_demand[j].x[i] for j in model.J) <= model.s[i] * model.y[i]
model.con_transportation = pe.Constraint(model.I, rule=con_transportation)

# objective
def obj_min_cost(model):
    return sum(model.f[i] * model.y[i] for i in model.I)\
        + sum(model.blk_demand[j].variable_cost for j in model.J)
model.obj_min_cost = pe.Objective(sense=pe.minimize, rule=obj_min_cost)

In [ ]:
result = solver.solve(model)
model.display()

# Implementation with Blocks Indexed by Supply Sites

In [ ]:
with open(cfl_blk_supply_filename) as fh:
    data = json.load(fh)
pprint(data)

In [ ]:
model = pe.ConcreteModel("Blocks (by Supply Site)")

# sets
model.I = pe.Set(initialize=data['supply_sites'])
model.J = pe.Set(initialize=data['demand_sites'])

# parameters (not indexed in I)
model.d = pe.Param(model.J, initialize=data['demand'])

# blocks (indexed in I)
def blk_supply(block, i):
    blk_data = data['supply_data'][i]
    J = block.model().J # borrow the set of demand sites from the overarching model
    block.c = pe.Param(J, initialize=blk_data['variable_cost'])
    block.f = pe.Param(initialize=blk_data['fixed_cost'])
    block.s = pe.Param(initialize=blk_data['supply'])
    block.x = pe.Var(J, domain=pe.NonNegativeReals)
    block.y = pe.Var(domain=pe.Binary)
    transportation = sum(block.x[j] for j in J)
    block.con_transportation = pe.Constraint(expr=(transportation <= block.s * block.y))
    block.obj = block.f * block.y + sum(block.c[j] * block.x[j] for j in J)
model.blk_supply = pe.Block(model.I, rule=blk_supply)

# constraints (not indexed in I)
def con_satisfaction(model, j):
    return sum(model.blk_supply[i].x[j] for i in model.I) == model.d[j]
model.con_satisfaction = pe.Constraint(model.J, rule=con_satisfaction)

# objective
def obj_min_cost(model):
    return sum(model.blk_supply[i].obj for i in model.I)
model.obj_min_cost = pe.Objective(sense=pe.minimize, rule=obj_min_cost)

In [ ]:
result = solver.solve(model)
model.display()